In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
import tensorflow as tf

tf.debugging.set_log_device_placement(True)

# Пример операции для проверки устройств
a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
b = tf.constant([[1.0, 1.0], [0.0, 1.0]])
c = tf.matmul(a, b)

print(c)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[1. 3.]
 [3. 7.]], shape=(2, 2), dtype=float32)


In [4]:
import tensorflow as tf
print(tf.__version__)

2.10.0


In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import itertools
from tensorflow.keras.models import load_model

In [2]:
data = pd.read_csv('hand_landmarks_dataset.csv')

In [3]:
X, y = data.drop(columns=['label','label_id', 'path']), data['label_id']

In [4]:
def super_frame(data, sequence_length = 37, num_features = 63):
    raw_data = data.to_numpy()
    n_sequences = len(raw_data) // sequence_length
    raw_data = raw_data[:n_sequences * sequence_length]
    X_seq = raw_data.reshape((n_sequences, sequence_length, num_features))

    # "Суперкадр" — среднее по последовательности
    X_mean = np.mean(X_seq, axis=1, keepdims=True)  # (n_sequences, 1, 63)

    # Добавляем к последовательности
    X_augmented = np.concatenate([X_seq, X_mean], axis=1)  # (n_sequences, 38, 63)

    X = pd.DataFrame(X_augmented.reshape(38*n_sequences, 63))
    
    return X


In [5]:
prep_X = super_frame(X)

In [6]:
minmax = MinMaxScaler()
X_scaled = minmax.fit_transform(prep_X)

window_size_x = 38
window_size_y = 37
n_samples = len(X_scaled) // window_size_x
n_samples_y = len(y) // window_size_y

y_seq = np.array(y[:n_samples_y * window_size_y]).reshape(n_samples_y, window_size_y)

y_seq = y_seq[:, 0] 

X_scaled = X_scaled.reshape(n_samples, 38, 63)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_seq, train_size=0.8, random_state=13)

In [7]:
y_train = to_categorical(y_train, num_classes=27)
y_test = to_categorical(y_test, num_classes=27)

In [8]:
for i in range(0, 6):
    model = load_model(f'model_{i}_best.h5')
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Model: {i}, Accuracy: {test_acc:.4f}, Loss: {test_loss:.4f}")

Model: 0, Accuracy: 0.7095, Loss: 1.0160


Model: 1, Accuracy: 0.7428, Loss: 0.9192


Model: 2, Accuracy: 0.7449, Loss: 0.9271


Model: 3, Accuracy: 0.7462, Loss: 0.9124


Model: 4, Accuracy: 0.7368, Loss: 0.9721
Model: 5, Accuracy: 0.7487, Loss: 0.9075
